## Week-04 assignment -- House Price Prediction 

#### Submitted by Gururaj H C 

In [1]:
# Importing the necessary libraries
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')
print('Done Importing the libraries')

Done Importing the libraries


In [2]:
#Loading the train dataset
df = pd.read_csv('train.csv')
df.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


### Data Preprocessing 

In [3]:
#Checking the scale of features
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [4]:
#Checking the Datatype and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
#Identifying the columns with null values
df.isnull().sum().sort_values(ascending=False).head(20)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
MasVnrType         8
Electrical         1
Id                 0
dtype: int64

In [6]:
#Fixing the null values
df['PoolQC'] = df['PoolQC'].fillna(value='NP')
df['MiscFeature'] = df['MiscFeature'].fillna(value='Not_present')
df['Alley'] = df['Alley'].fillna(value='No_access')
df['Fence'] = df['Fence'].fillna(value='No_fence')
df['FireplaceQu'] = df['FireplaceQu'].fillna(value='No_fire')
df['LotFrontage'] = df['LotFrontage'].fillna(value=0.0)
df['GarageFinish'] = df['GarageFinish'].fillna(value='No_garage')
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(value=df['GarageYrBlt'].mode()[0])
df['GarageQual'] = df['GarageQual'].fillna(value='No_garage')
df['GarageType'] = df['GarageType'].fillna(value='No_garage')
df['GarageCond'] = df['GarageCond'].fillna(value='No_garage')
df['BsmtFinType2'] = df['BsmtFinType2'].fillna(value='No_basement')
df['BsmtExposure'] = df['BsmtExposure'].fillna(value='No_basement')
df['BsmtFinType1'] = df['BsmtFinType1'].fillna(value='No_basement')
df['BsmtQual'] = df['BsmtQual'].fillna(value='No_basement')
df['BsmtCond'] = df['BsmtCond'].fillna(value='No_basement')
df['MasVnrArea'] = df['MasVnrArea'].fillna(value=0.0)
df['MasVnrType'] = df['MasVnrType'].fillna(value='No_masonary')
df['Electrical'] = df['Electrical'].fillna(value=df['GarageYrBlt'].mode()[0])
df['LotFrontage'] = df['LotFrontage'].astype(int)
df['MasVnrArea'] = df['MasVnrArea'].astype(int)
df['GarageYrBlt'] = df['GarageYrBlt'].astype(int)

In [7]:
#Treating the outliers
dtypes = pd.DataFrame(df.dtypes)
dtypes = dtypes.reset_index(drop=False)
dtypes = dtypes.rename(columns={'index':'Feature',0:'dtype'})
cols = list(dtypes['Feature'][dtypes['dtype'] == 'int64'])
to_remove = ['Id','LowQualFinSF','SalePrice','BsmtHalfBath','EnclosedPorch','3SsnPorch','ScreenPorch',
             'PoolArea','MiscVal','BsmtFinSF2','KitchenAbvGr']

for i in to_remove:
    if i in cols:
        cols.remove(i)
for i in cols:
    df = df.sort_values(by=i, ascending=True)
    Q1 = df[i].quantile(0.25)
    Q3 = df[i].quantile(0.75)
    IQR = Q3 - Q1
    ll = Q1 - 1.5*IQR
    ul = Q3 + 1.5*IQR
    for j in range(len(df)):
        if df[i][j] < ll:
            df[i][j] = ll
        elif df[i][j] > ul:
            df[i][j] = ul

In [8]:
#Nominal encoding for the categorical variables
df_dt = pd.DataFrame(df.dtypes)
df_dt = df_dt.reset_index(drop=False)
df_dt = df_dt.rename(columns = {'index':'Name',0:'Type'})
cols2 = list(df_dt['Name'][df_dt['Type'] == 'object'])
df_res = pd.get_dummies(df, columns=cols2,drop_first=True)

In [9]:
# Scaling and then appplying the dimensionality reduction technique
ss = StandardScaler()
y = np.log(df_res['SalePrice'])
X = df_res.drop(columns='SalePrice', axis=1)
X_Scaled = ss.fit_transform(X)
pca = PCA(n_components=25)
X_pca = pca.fit_transform(X_Scaled)
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=30)

In [10]:
# Using Grid search identified the best performing models and corresponding hyper-parameters
# Applying Cross-validation
random_forest_model = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42)
xgboost_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
random_forest_scores = cross_val_score(random_forest_model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')
random_forest_rmse_scores = [np.sqrt(-score) for score in random_forest_scores]
average_random_forest_rmse = np.mean(random_forest_rmse_scores)
xgboost_scores = cross_val_score(xgboost_model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')
xgboost_rmse_scores = [np.sqrt(-score) for score in xgboost_scores]
average_xgboost_rmse = np.mean(xgboost_rmse_scores)
print("Random Forest Average RMSE:", average_random_forest_rmse)
print("XGBoost Average RMSE:", average_xgboost_rmse)

Random Forest Average RMSE: 0.16542589206305208
XGBoost Average RMSE: 0.16288769158191552


#### Performance of both the models is quite similar. That being said, XGBoost has slightly better RMSE. Hence choosing to go with it

### Now we are done with training the model, now lets load the test dataset 

#### Performing the same pre-processing steps on the test dataset

In [11]:
df2 = pd.read_csv('test.csv')
df2['PoolQC'] = df2['PoolQC'].fillna(value='NP')
df2['MiscFeature'] = df2['MiscFeature'].fillna(value='Not_present')
df2['Alley'] = df2['Alley'].fillna(value='No_access')
df2['Fence'] = df2['Fence'].fillna(value='No_fence')
df2['FireplaceQu'] = df2['FireplaceQu'].fillna(value='No_fire')
df2['LotFrontage'] = df2['LotFrontage'].fillna(value=0.0)
df2['GarageFinish'] = df2['GarageFinish'].fillna(value='No_garage')
df2['GarageYrBlt'] = df2['GarageYrBlt'].fillna(value=df2['GarageYrBlt'].mode()[0])
df2['GarageQual'] = df2['GarageQual'].fillna(value='No_garage')
df2['GarageType'] = df2['GarageType'].fillna(value='No_garage')
df2['GarageCond'] = df2['GarageCond'].fillna(value='No_garage')
df2['BsmtFinType2'] = df2['BsmtFinType2'].fillna(value='No_basement')
df2['BsmtExposure'] = df2['BsmtExposure'].fillna(value='No_basement')
df2['BsmtFinType1'] = df2['BsmtFinType1'].fillna(value='No_basement')
df2['BsmtQual'] = df2['BsmtQual'].fillna(value='No_basement')
df2['BsmtCond'] = df2['BsmtCond'].fillna(value='No_basement')
df2['BsmtHalfBath'] = df2['BsmtHalfBath'].fillna(value=0.0)
df2['BsmtFullBath'] = df2['BsmtFullBath'].fillna(value=0.0)
df2['BsmtFinSF1'] = df2['BsmtFinSF1'].fillna(value=0.0)
df2['BsmtFinSF1'] = df2['BsmtFinSF1'].fillna(value=0.0)
df2['GarageCars'] = df2['GarageCars'].fillna(value=2.0)
df2['GarageArea'] = df2['GarageArea'].fillna(value=0.0)
df2['TotalBsmtSF'] = df2['TotalBsmtSF'].fillna(value=0.0)
df2['BsmtUnfSF'] = df2['BsmtUnfSF'].fillna(value=0.0)
df2['BsmtFinSF2'] = df2['BsmtFinSF2'].fillna(value=0.0)
df2['BsmtHalfBath'] = df2['BsmtHalfBath'].astype(int)
df2['BsmtFullBath'] = df2['BsmtFullBath'].astype(int)
df2['BsmtFinSF1'] = df2['BsmtFinSF1'].astype(int)
df2['BsmtFinSF1'] = df2['BsmtFinSF1'].astype(int)
df2['GarageCars'] = df2['GarageCars'].astype(int)
df2['GarageArea'] = df2['GarageArea'].astype(int)
df2['TotalBsmtSF'] = df2['TotalBsmtSF'].astype(int)
df2['BsmtUnfSF'] = df2['BsmtUnfSF'].astype(int)
df2['BsmtFinSF2'] = df2['BsmtFinSF2'].astype(int)
df2['MasVnrArea'] = df2['MasVnrArea'].fillna(value=0.0)
df2['MasVnrType'] = df2['MasVnrType'].fillna(value='No_masonary')
df2['Electrical'] = df2['Electrical'].fillna(value=df2['GarageYrBlt'].mode()[0])
df2['LotFrontage'] = df2['LotFrontage'].astype(int)
df2['MasVnrArea'] = df2['MasVnrArea'].astype(int)
df2['GarageYrBlt'] = df2['GarageYrBlt'].astype(int)
dtypes = pd.DataFrame(df2.dtypes)
dtypes = dtypes.reset_index(drop=False)
dtypes = dtypes.rename(columns={'index':'Feature',0:'dtype'})
cols = list(dtypes['Feature'][dtypes['dtype'] == 'int64'])
to_remove = ['Id','LowQualFinSF','SalePrice','BsmtHalfBath','EnclosedPorch','3SsnPorch','ScreenPorch',
             'PoolArea','MiscVal','BsmtFinSF2','KitchenAbvGr']

for i in to_remove:
    if i in cols:
        cols.remove(i)
for i in cols:
    df2 = df2.sort_values(by=i, ascending=True)
    Q1 = df2[i].quantile(0.25)
    Q3 = df2[i].quantile(0.75)
    IQR = Q3 - Q1
    ll = Q1 - 1.5*IQR
    ul = Q3 + 1.5*IQR
    for j in range(len(df2)):
        if df2[i][j] < ll:
            df2[i][j] = ll
        elif df2[i][j] > ul:
            df2[i][j] = ul
df2_dt = pd.DataFrame(df2.dtypes)
df2_dt = df2_dt.reset_index(drop=False)
df2_dt = df2_dt.rename(columns = {'index':'Name',0:'Type'})
cols2 = list(df2_dt['Name'][df2_dt['Type'] == 'object'])
df2_res = pd.get_dummies(df2, columns=cols2,drop_first=True)
ss = StandardScaler()
df2_scaled = ss.fit_transform(df2_res)
pca = PCA(n_components=25)
X_pca_test = pca.fit_transform(df2_scaled)
xgboost_model.fit(X_train,y_train)
y_pred_res = xgboost_model.predict(X_pca_test)
res2 = np.exp(y_pred_res)
res = res2.astype(int)

In [12]:
#Writing the result to sample_submission.csv file
col1 = df2['Id']
col2 = res.tolist()
col_names = ['Id', 'SalePrice']
sample_submission = pd.DataFrame({col_names[0]: col1, col_names[1]: col2})
sample_submission.reset_index(drop=True, inplace=True)
sample_submission.to_csv('sample_submission.csv', index=False)
print('sample_submission.csv file successfully created!')

sample_submission.csv file successfully created!


#### Checking the result by loading the csv file


In [13]:
res3 = pd.read_csv('sample_submission.csv')
res3.head()

,Id,SalePrice
0,2914,384321
1,2703,195764
2,2633,132370
3,2855,129994
4,2616,191763
